In [1]:
import numpy as np
from tensorflow.keras import models, layers

# XOR 데이터셋
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,0])  # XOR 결과

# 모델 생성
model = models.Sequential([
    layers.Dense(4, input_dim=2, activation='relu'),  # 은닉층
    layers.Dense(1, activation='sigmoid')            # 출력층
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 학습
model.fit(X, y, epochs=100, verbose=0)

# 평가
print("정확도:", model.evaluate(X, y, verbose=0)[1])

# 예측
print("예측 결과:", model.predict(X).round())


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


정확도: 0.75
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
예측 결과: [[0.]
 [1.]
 [1.]
 [1.]]


### 합성곱 연산 이해

In [2]:
import numpy as np

# 간단한 5×5 이미지
image = np.array([
    [1, 1, 1, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 0],
    [0, 1, 1, 0, 0]
])

# 3×3 수직선 감지 필터
vertical_filter = np.array([
    [1,  0, -1],
    [1,  0, -1],
    [1,  0, -1]
])

# 합성곱 연산 (간단 버전)
def simple_convolution(image, filter):
    output = []
    # 필터가 이미지 위를 슬라이딩
    for i in range(image.shape[0] - 2):
        row = []
        for j in range(image.shape[1] - 2):
            # 이미지의 작은 영역 추출
            region = image[i:i+3, j:j+3]
            # 필터와 원소별 곱셈 후 합산
            value = np.sum(region * filter)
            row.append(value)
        output.append(row)
    return np.array(output)

# 합성곱 수행
result = simple_convolution(image, vertical_filter)

print("원본 이미지:")
print(image)
print("\n수직선 감지 필터:")
print(vertical_filter)
print("\n결과 (특징 맵):")
print(result)

원본 이미지:
[[1 1 1 0 0]
 [0 1 1 1 0]
 [0 0 1 1 1]
 [0 0 1 1 0]
 [0 1 1 0 0]]

수직선 감지 필터:
[[ 1  0 -1]
 [ 1  0 -1]
 [ 1  0 -1]]

결과 (특징 맵):
[[-2  0  2]
 [-3 -2  2]
 [-3 -1  2]]


PyTorch로 간단한 CNN만들기

In [3]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    """간단한 CNN 모델 - 이미지 분류용"""

    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()

        # 특징 추출 부분
        self.features = nn.Sequential(
            # 첫 번째 합성곱 블록
            nn.Conv2d(
                in_channels=3,      # 입력: RGB 3채널
                out_channels=16,    # 출력: 16개 필터
                kernel_size=3,      # 3×3 필터
                padding=1           # 크기 유지
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 크기 절반

            # 두 번째 합성곱 블록
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 세 번째 합성곱 블록
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 분류 부분
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 128),  # 완전연결층
            nn.ReLU(),
            nn.Dropout(0.5),              # 과적합 방지
            nn.Linear(128, num_classes)   # 출력층
        )

    def forward(self, x):
        # 순전파
        x = self.features(x)      # 특징 추출
        x = self.classifier(x)    # 분류
        return x

# 모델 생성
model = SimpleCNN(num_classes=10)

# 모델 구조 출력
print(model)

# 예시 입력 (배치크기=1, RGB, 32×32)
sample_input = torch.randn(1, 3, 32, 32)
output = model(sample_input)

print(f"\n입력 크기: {sample_input.shape}")
print(f"출력 크기: {output.shape}")
print(f"출력: {output}")

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1024, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)

입력 크기: torch.Size([1, 3, 32, 32])
출력 크기: torch.Size([1, 10])
출력: tensor([[ 0.1051, -0.0713,  0.0583, -0.0324,  0.0643, -0.0638, -0.0944, -0.0920,
         -0.0902, -0.0381]], grad

## 실전 이미지 분류 (CIFAR-10)

In [9]:
# ✅ PyTorch 빠른 학습용 경량 CNN (CIFAR-10, tqdm + 에포크 요약)
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

# 0) 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

# 1) 전처리(가벼운 증강 + 정규화)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

# 2) CIFAR-10 로드 + 부분 데이터(10k)
train_full = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
subset_indices = list(range(10_000))  # ← 10k개만 사용(속도↑)
trainset = Subset(train_full, subset_indices)

trainloader = DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=0, pin_memory=(device=="cuda")
)

# 3) 경량 CNN (채널 수 축소)
class TinyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.BatchNorm2d(16), nn.ReLU(),
            nn.MaxPool2d(2),  # 16x16
            nn.Conv2d(16, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2),  # 8x8
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1)),    # 1x1
            nn.Flatten(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.net(x)

model = TinyCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 4) 학습 루프 (tqdm 진행 표시 + 에포크 요약 출력)
def train_one_epoch(epoch):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    loop = tqdm(enumerate(trainloader), total=len(trainloader), leave=False)
    for i, (inputs, labels) in loop:
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, pred = outputs.max(1)
        total += labels.size(0)
        correct += pred.eq(labels).sum().item()

        # 진행 중 상태 갱신
        loop.set_description(f"에포크 [{epoch}]")
        loop.set_postfix(loss=loss.item(), acc=100.0*correct/total)

    # ✅ 에포크 종료 후 한 줄 요약 출력
    epoch_loss = running_loss / len(trainloader)
    epoch_acc  = 100.0 * correct / total
    print(f"에포크 [{epoch}] 손실: {epoch_loss:.3f} | 정확도: {epoch_acc:.2f}%")

# 5) 학습 실행
print("학습 시작!")
for epoch in range(1, 5 + 1):  # ← 5에포크
    train_one_epoch(epoch)
print("완료!")


device: cuda
학습 시작!


에포크 [1] 손실: 1.979 | 정확도: 28.13%


에포크 [2] 손실: 1.739 | 정확도: 37.23%


에포크 [3] 손실: 1.623 | 정확도: 41.73%


에포크 [4] 손실: 1.542 | 정확도: 44.28%


에포크 [5] 손실: 1.472 | 정확도: 47.02%
완료!
